# Final Project

## Part 1

The cell below is for imports. You are only allowed to use `numpy` for this part.No additional imports are allowed

In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp39-cp39-linux_x86_64.whl size=3380634 sha256=98257263e2067f49c97503b51b9767166452fdbc3955d3c0485de5c60516a01a
  Stored in directory: /root/.cache/pip/wheels/db/81/9f/43a031fce121c845baca1c5d9a1468cad98208286aa2832de9
Successfully built mpi4py


In [2]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import petsc4py
else:
    try:
        import petsc4py
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-real.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-complex.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        import petsc4py

In [4]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc

comm   = MPI.COMM_WORLD
nprocs = comm.Get_size()
rank   = comm.Get_rank()

Now, we will create our class `SparseMatrix`

The class will represent a sparse matrix in `COO` format.\
it should also keep track of the shape of the matrix.\
You need to add the necessary attributes to your class to account for the aforementioned requirements

Let's start with the `__init__` method of our class:\
it should take one additional argument `arg` that will represent the various objects from which we can instantiate our class.

First, we should be able to construct an instance of our class from a regular `numpy` 2d array. \
Inside the `__init__` method, check if `arg` is an instance of a `numpy` array. \
Then, check if the provided array represents a valid matrix.\
If it is not the case, an exception should be raised 

In [5]:
class SparseMatrix:    
    def __init__(self, arg):
        if isinstance(arg, np.ndarray):
            if arg.ndim != 2:
                raise ValueError("Input array must be 2D")
            self.data = arg[np.nonzero(arg)] # get nonzero elements of the array
            self.row_indices, self.col_indices = np.nonzero(arg) # get row and column indices of nonzero elements
            self.shape = arg.shape
        else:
            raise TypeError("Input must be a numpy array")
        

Next, we should be able to construct an instance of our class from a tuple of 3 `numpy` arrays representing a matrix in `COO` format (x, Y, Values)\
Extend the __init__ method by checking if arg is an instance of this case.
Then, check if the provided array represents a valid matrix.
If it is not the case, an exception should be raised

In [6]:
class SparseMatrix:
    def __init__(self, arg):
        if isinstance(arg, np.ndarray):
            if arg.ndim != 2:
                raise ValueError("Input array must be 2D")
            self.data = arg[np.nonzero(arg)] # get nonzero elements of the array
            self.row_indices, self.col_indices = np.nonzero(arg) # get row and column indices of nonzero elements
            self.shape = arg.shape
        elif isinstance(arg, tuple) and len(arg) == 3:
            if not all(isinstance(arr, np.ndarray) for arr in arg):
                raise TypeError("All elements of the input tuple must be numpy arrays")
            if not all(arr.ndim == 1 for arr in arg):
                raise ValueError("All arrays in the input tuple must be 1D")
            if not all(arr.shape == arg[0].shape for arr in arg):
                raise ValueError("All arrays in the input tuple must have the same shape")
            self.row_indices, self.col_indices, self.data = arg
            self.shape = arg[0].shape
        else:
            raise TypeError("Input must be a numpy array or a tuple of three numpy arrays")


Create a function `cooTranspose` that takes an instance of our class `SparseMatrix` and returns its transpose.

In [7]:
def cooTranspose(a):
    transposed_indices = np.vstack((a.col_indices, a.row_indices))
    transposed_data = a.data
    transposed_shape = tuple(reversed(a.shape))
    return SparseMatrix((transposed_indices[0], transposed_indices[1], transposed_data))

Create a function `cooMatVec` that takes an instance of our class `SparseMatrix` and a vector \ 
as a `numpy` array and returns their product.

In [8]:
def cooMatVec(A, x):
    if A.shape[1] != x.shape[0]:
        raise ValueError("Matrix and vector dimensions do not match")
    
    # Initialize the result vector
    result = np.zeros(A.shape[0])
    
    # Loop over the non-zero elements of the matrix and add their contributions to the result vector
    for i in range(A.data.shape[0]):
        result[A.row_indices[i]] += A.data[i] * x[A.col_indices[i]]
    
    return result


create a function cooMatMat that takes two instances of our class `SparseMatrix` and \ 
returns their matrix product as a 2 dimentional numpy array

In [9]:
def cooMatMat(A, B):
    if A.shape[1] != B.shape[0]:
        raise ValueError("Matrix dimensions do not match for matrix multiplication")

    # Initialize the result matrix
    result = np.zeros((A.shape[0], B.shape[1]))

    # Loop over the non-zero elements of the first matrix
    for i in range(A.data.shape[0]):
        row = A.row_indices[i]
        col = A.col_indices[i]
        value = A.data[i]

        # Loop over the non-zero elements of the second matrix with matching column index
        for j in range(B.data.shape[0]):
            if B.row_indices[j] == col:
                result[row, B.col_indices[j]] += value * B.data[j]

    return result

In [10]:
# Define a test matrix
A = np.array([[1, 0, 2], [0, 3, 0], [4, 0, 5]])

# Create a SparseMatrix instance from the test matrix
A_sparse = SparseMatrix(A)

# Print the attributes of the SparseMatrix instance
print("Sparse matrix data:", A_sparse.data)
print("Sparse matrix row indices:", A_sparse.row_indices)
print("Sparse matrix column indices:", A_sparse.col_indices)
print("Sparse matrix shape:", A_sparse.shape)

# Transpose the SparseMatrix instance
A_transpose = cooTranspose(A_sparse)

# Print the attributes of the transposed SparseMatrix instance
print("Transposed sparse matrix data:", A_transpose.data)
print("Transposed sparse matrix row indices:", A_transpose.row_indices)
print("Transposed sparse matrix column indices:", A_transpose.col_indices)
print("Transposed sparse matrix shape:", A_transpose.shape)

# Define a test vector
x = np.array([1, 2, 3])

# Compute the product of the SparseMatrix instance and the test vector
Ax = cooMatVec(A_sparse, x)

# Print the product of the SparseMatrix instance and the test vector
print("Product of sparse matrix and vector:", Ax)

# Define a second test matrix
B = np.array([[1, 0], [0, 2], [3, 0]])

# Create a SparseMatrix instance from the second test matrix
B_sparse = SparseMatrix(B)

# Compute the product of the two SparseMatrix instances
AB = cooMatMat(A_sparse, B_sparse)

# Print the product of the two SparseMatrix instances
print("Product of sparse matrices: ")
print(AB)


Sparse matrix data: [1 2 3 4 5]
Sparse matrix row indices: [0 0 1 2 2]
Sparse matrix column indices: [0 2 1 0 2]
Sparse matrix shape: (3, 3)
Transposed sparse matrix data: [1 2 3 4 5]
Transposed sparse matrix row indices: [0 2 1 0 2]
Transposed sparse matrix column indices: [0 0 1 2 2]
Transposed sparse matrix shape: (5,)
Product of sparse matrix and vector: [ 7.  6. 19.]
Product of sparse matrices: 
[[ 7.  0.]
 [ 0.  6.]
 [19.  0.]]


In [11]:
A = np.array([[1, 0, 2], [0, 3, 0], [4, 0, 5]])
B = np.array([[1, 0], [0, 2], [3, 0]])
x = np.array([1, 2, 3])
print(A@B)
print()
print(A@x)

[[ 7  0]
 [ 0  6]
 [19  0]]

[ 7  6 19]


# Part 2

In this part, we will be solving a system of linear equations involving a sparse matrix `A` in parallel. \
You will not have to solve the system. However, you will have to implement the function `CreateLocalMatVec` that sets the system for the class `LinearSystem`. \
At the end, compare the results and explain any discrepancies.

In [12]:
#Create the matrix and Rhs
np.random.seed(42)
from scipy.sparse import random
if rank == 0:
    n = 100
    # Set parameters for the sparse matrix
    density = 0.3 # density of non-zero elements (between 0 and 1)
    A =  random(n, n, density=density, format='csr')
    x = np.random.rand(n)
    B_all = np.dot(A.toarray(),x)
else:
    A = None
    B_all = None

In [13]:
def CreateLocalMatVec(A, B_all):
    if rank == 0:
        shape = A.shape
        nrows = shape[0]
        # split the number of rows evenly (as possible) among the MPI tasks
        N_pertask, extra = divmod(nrows, nprocs)
    
        # count: the size of each sub-task
        count = [N_pertask + 1 if i < extra else N_pertask for i in range(nprocs)]

        # displacement: the starting index of each sub-task
        displ = [sum(count[:i]) for i in range(nprocs)]

        #---- Send the relevant subsets of A and B to each slave MPI task ----
        for i in range(1,nprocs):
    
            # Get the start and end row index for this MPI task
            rstart = displ[i]
            rend   = rstart + count[i]
    
            #---- Get the subsets of A and B using these rows ----
            A_indptr  = A.indptr[rstart:rend+1] - A.indptr[rstart]
            pstart    = A.indptr[rstart]
            pend      = A.indptr[rend]
    
            A_indices = A.indices[pstart:pend]
            A_data    = A.data[pstart:pend]
            B         = B_all[rstart:rend]
    
            # Save the lengths of each array
            lengths = {
                    'A_indptr' : len(A_indptr),
                    'A_indices': len(A_indices),
                    'A_data'   : len(A_data),
                    'B'        : len(B),
                    }
    
            # Send the arrays and their lenghts to the relevant MPI task
            comm.send(lengths, dest=i, tag=1)
            comm.Send(A_indptr,  dest=i, tag=2)
            comm.Send(A_indices, dest=i, tag=3)
            comm.Send(A_data,    dest=i, tag=4)
            comm.Send(B,         dest=i, tag=5)
    
        #---- Set the relevant subsets of A and B for the master MPI task (we don't need to do an MPI Send)
        rstart = displ[0]
        rend   = rstart + count[0]
    
        A_indptr  = A.indptr[rstart:rend+1] - A.indptr[rstart]
        pstart    = A.indptr[rstart]
        pend      = A.indptr[rend]
    
        A_indices = A.indices[pstart:pend]
        A_data    = A.data[pstart:pend]
        B         = B_all[rstart:rend]
    
    else:
        # Receive the array lengths
        lengths   = comm.recv(source=0, tag=1)
        # Initialise the buffers
        A_indptr  = np.empty(lengths['A_indptr'], dtype=np.intc)
        A_indices = np.empty(lengths['A_indices'], dtype=np.intc)
        A_data    = np.empty(lengths['A_data'], dtype=np.float64)
        B         = np.empty(lengths['B'], dtype=np.float64)
        # Receive the arrays
        comm.Recv(A_indptr,  source=0, tag=2)
        comm.Recv(A_indices, source=0, tag=3)
        comm.Recv(A_data,    source=0, tag=4)
        comm.Recv(B,         source=0, tag=5)
    
        shape = None
    
    #broadcast shape
    shape = comm.bcast(shape, root=0)
        
    return (A_indptr, A_indices, A_data, B, shape)


In [14]:
class LinearSystem():
    def __init__(self, A=None, shape=None, rhs=None, solver=None, comm=None):
        
        #from petsc4py import PETSc
        self.opts = PETSc.Options()

        self.ksp = PETSc.KSP()
        self.ksp.create()
        
        mat = PETSc.Mat().createAIJ(comm=comm, size=shape, csr=A)
        mat.setUp()
        mat.assemblyBegin()
        mat.assemblyEnd()
        self.sol, self.rhs = mat.getVecs()
        self.rhs.setArray(rhs)
        
        
        #---- Set up solver -----
        self.ksp = PETSc.KSP().create(comm=comm)
        
        # It is commonly used with the direct solver preconditioners like PCLU and PCCHOLESKY
        self.ksp.setType('preonly')
        pc = self.ksp.getPC()
        pc.setType('lu')
        pc.setFactorSolverType(solver)
        
        self.ksp.setOperators(mat)
        self.ksp.setFromOptions() # Apply any command line options
        self.ksp.setUp()
    
    def solve(self): 
        # st=timeit.default_timer()
        self.ksp.solve(self.rhs, self.sol)

In [15]:
(A_indptr, A_indices, A_data, b, shape) = CreateLocalMatVec(A, B_all)

L = LinearSystem(A=(A_indptr, A_indices, A_data), rhs=b, shape=shape, solver="mumps")
ts = MPI.Wtime()
L.solve()
te = MPI.Wtime()
tt = comm.reduce(te - ts, op=MPI.MAX, root=0)
if rank == 0:
    print("Timing to solve the linear system with petsc", tt)

Timing to solve the linear system with petsc 0.0006531859999938661


In [16]:
#---- Gather the solution onto a single array on the master MPI task
if rank == 0:
    X = np.empty(shape[0],dtype=np.double)
else:
    X = None
comm.Gatherv(L.sol.array,X)

In [18]:
np.random.seed(42)

if rank == 0:
    B = SparseMatrix(A.toarray())
    
    from scipy import sparse
    from scipy.sparse.linalg import spsolve
    mat = sparse.csr_matrix((B.data, (B.row_indices, B.col_indices)))
    ts = MPI.Wtime()
    sol = spsolve(mat, b)
    te = MPI.Wtime()
    print("Timing to solve the linear system with spsolve", te - ts)

Timing to solve the linear system with spsolve 0.0033241539999977476


In [19]:
if rank == 0:
    print(np.allclose(X,sol))

True


Put your response below:

**The two methods use different solvers to solve the linear system. The first method uses the MUMPS direct solver through the PETSc library, while the second method uses the spsolve function from the SciPy library, which is based on the SuperLU direct solver**.

**The MUMPS solver is generally faster and more efficient for solving sparse linear systems with symmetric positive-definite matrices, while SuperLU is designed to work with general sparse matrices.**

**The PETSc method has a timing of 0.000653 seconds, while the SciPy method has a timing of 0.00332 seconds, indicating that the PETSc method is faster.**

**However, both methods produce the same solution, as confirmed by the np.allclose comparison, which returns True. Therefore, the discrepancies in timing are due to differences in the algorithms and libraries used, and not due to differences in the solutions produced.**

